In [56]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV #подбор гиперпараметров по сетке
from sklearn.model_selection import RandomizedSearchCV #рандомизированный подбор гиперпараметров

#Импорт библиотек для подбора гиперпараметров с помощью Hyperopt
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

import optuna #подбор гиперпараметров по Оптуна

%matplotlib inline
plt.style.use('seaborn')

In [57]:
data = pd.read_csv('data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [58]:
X = data.drop('Activity', axis=1)
y = data['Activity']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

#### **1. Логистическая регрессия**

Зафиксируем метрики, которые были получены без дополнительной настройки, т.е со значениями гиперпараметров, установленных по умолчанию:

In [60]:
# Создаем модель
log_reg = linear_model.LogisticRegression(max_iter=50)
# Обучаем модель
%time log_reg.fit(X_train, y_train)
# Делаем предсказания для тренировочной и тестовой выборки
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)
print('Значение f1-score на тренировочной выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('Значение f1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


CPU times: total: 1.48 s
Wall time: 559 ms
Значение f1-score на тренировочной выборке: 0.88
Значение f1-score на тестовой выборке: 0.78


c:\Users\Пользователь\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


****GridSearchCV****

In [61]:
# задаем сетку гиперпараметров для логистической регрессии
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

In [62]:
# Подбор гиперпараметров по сетке
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_grid=param_grid, #сетка параметров
    cv=5, #количество фолдов для кросс-валидации
    n_jobs = -1
)

%time grid_search.fit(X_train, y_train)
#Делаем предсказания
y_train_pred_gs = grid_search.predict(X_train)
y_test_pred_gs = grid_search.predict(X_test)
print('Значение f1-score на тренировочной выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred_gs)))
print('Значение f1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_gs)))

CPU times: total: 5.61 s
Wall time: 3min 34s
Значение f1-score на тренировочной выборке: 0.83
Значение f1-score на тестовой выборке: 0.79


c:\Users\Пользователь\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


****RandomizedSearchCV****

In [63]:
# Задаем сетку параметров
param_grid = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},

random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_grid, #сетка параметров
    cv=5, #количество фолдов для кросс-валидации
    n_iter = 10, #количество комбинаций
    n_jobs = -1
)

%time random_search.fit(X_train, y_train)
#Делаем предсказания
y_train_pred_rs = random_search.predict(X_train)
y_test_pred_rs = random_search.predict(X_test)
print('Значение f1-score на тренировочной выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred_rs)))
print('Значение f1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rs)))


CPU times: total: 1.77 s
Wall time: 30.6 s
Значение f1-score на тренировочной выборке: 0.87
Значение f1-score на тестовой выборке: 0.78


c:\Users\Пользователь\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### **2. Случайный лес**

Зафиксируем метрики, которые были получены без дополнительной настройки, т.е со значениями гиперпараметров, установленных по умолчанию:

In [64]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
%time rf.fit(X_train, y_train)
#Выводим значения метрики
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)
print('Значение f1-score на тренировочной выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('Значение f1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

CPU times: total: 2.3 s
Wall time: 2.32 s
Значение f1-score на тренировочной выборке: 1.00
Значение f1-score на тестовой выборке: 0.80


****GridSearchCV****

In [65]:
# Зададаем сетку параметров
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }

In [66]:
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  

%time grid_search_forest.fit(X_train, y_train)
y_train_pred_rf_gs = grid_search_forest.predict(X_train)
y_test_pred_rf_gs = grid_search_forest.predict(X_test)
print('Значение f1-score на тренировочной выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred_rf_gs)))
print('Значение f1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_gs)))


CPU times: total: 2.78 s
Wall time: 1min 13s
Значение f1-score на тренировочной выборке: 0.94
Значение f1-score на тестовой выборке: 0.80


****RandomizedSearchCV****

In [67]:
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_grid, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train)
y_train_pred_rf_rs = random_search_forest.predict(X_train)
y_test_pred_rf_rs = random_search_forest.predict(X_test)
print('Значение f1-score на тренировочной выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred_rf_rs)))
print('Значение f1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_rs)))

CPU times: total: 3.06 s
Wall time: 35.7 s
Значение f1-score на тренировочной выборке: 0.95
Значение f1-score на тестовой выборке: 0.81


****Hyperopt****

In [68]:
# задаем сетку параметров
space = {'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [69]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    score = cross_val_score(model, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [70]:
# подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )

print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 50/50 [04:41<00:00,  5.62s/trial, best loss: -0.8201982028293425]
Наилучшие значения гиперпараметров {'max_depth': 15.0, 'min_samples_leaf': 2.0, 'n_estimators': 142.0}


In [71]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

model.fit(X_train, y_train)
y_train_pred_rf_ho = model.predict(X_train)
y_test_pred_rf_ho = model.predict(X_test)
print('Значение f1-score на тренировочной выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred_rf_ho)))
print('Значение f1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_ho)))

Значение f1-score на тренировочной выборке: 0.98
Значение f1-score на тестовой выборке: 0.81


****Optuna****

In [72]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [73]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2022-07-25 14:50:58,383] A new study created in memory with name: RandomForestClassifier
[I 2022-07-25 14:51:00,412] Trial 0 finished with value: 0.9576090271424215 and parameters: {'n_estimators': 105, 'max_depth': 20, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.9576090271424215.
[I 2022-07-25 14:51:02,593] Trial 1 finished with value: 0.918820310124658 and parameters: {'n_estimators': 136, 'max_depth': 17, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9576090271424215.
[I 2022-07-25 14:51:05,345] Trial 2 finished with value: 0.9439024390243903 and parameters: {'n_estimators': 155, 'max_depth': 25, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9576090271424215.
[I 2022-07-25 14:51:07,001] Trial 3 finished with value: 0.8862530413625305 and parameters: {'n_estimators': 118, 'max_depth': 12, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.9576090271424215.
[I 2022-07-25 14:51:09,887] Trial 4 finished with value: 0.9219556635286973 and parameters: {'n_

CPU times: total: 2min 46s
Wall time: 2min 50s


In [74]:
# Обучим модель и рассчитаем метрики
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)

y_train_pred_rf_ot = model.predict(X_train)
y_test_pred_rf_ot = model.predict(X_test)
print('Значение f1-score на тренировочной выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred_rf_ot)))
print('Значение f1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_ot)))

Значение f1-score на тренировочной выборке: 0.99
Значение f1-score на тестовой выборке: 0.80
